In [3]:
#Loading packages
%pip install pandas seaborn matplotlib scikit-learn numpy openpyxl pipenv pycombat

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 11.6/11.6 MB 9.5 MB/s eta 0:00:00
     -------------------------------------- 294.9/294.9 kB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 8.0/8.0 MB 10.2 MB/s eta 0:00:00
     --------------------------------------- 11.0/11.0 MB 14.5 MB/s eta 0:00:00
     --------------------------------------- 15.8/15.8 MB 14.5 MB/s eta 0:00:00
     ------------------------------------- 251.3/251.3 kB 15.1 MB/s eta 0:00:00
     ---------------------------------------- 3.1/3.1 MB 10.4 MB/s eta 0:00:00
     ------------------------------------- 505.5/505.5 kB 10.5 MB/s eta 0:00:00
     -------------------------------------- 345.4/345.4 kB 7.1 MB/s eta 0:00:00
     -------------------------------------- 188.2/188.2 kB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 10.0 MB/s eta 0:00:00
     ---------------------------------------- 56.1/56.1 kB 3.1 MB/s eta 0:00:00
     --------------------------------------

In [6]:
data = pd.read_excel("01. Master_Latest data_Control clones_LP.xlsx", engine='openpyxl')
data_extracted_features = pd.read_excel("01. Master_Latest data_Control clones_LP.xlsx", usecols=['Batch number', 'Genotype Neuron', 'Div calculated', 'Culture treatment', 'Capacitance', 'Input Resistance', 'Resting membrane potential ', 'Maximum firing ', 'Rheobase', 'EPSC freq'])

print(data_extracted_features)

FileNotFoundError: [Errno 2] No such file or directory: 'data/01. Master_Latest data_Control clones_LP.xlsx'

In [ ]:
#Make GLM imputer work

from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import os

# Assuming 'data_extracted_features' is your input dataframe
data_extracted_features_impute_glm = data_extracted_features.copy()

# Handle 'EPSC freq' values
mask_glm = data_extracted_features_impute_glm['EPSC freq'].notna() & data_extracted_features_impute_glm['EPSC freq'].str.contains(">")
data_extracted_features_impute_glm.loc[mask_glm, 'EPSC freq'] = np.nan

# Convert 'EPSC freq' to numeric, setting errors='coerce' to handle any unexpected values
data_extracted_features_impute_glm['EPSC freq'] = pd.to_numeric(data_extracted_features_impute_glm['EPSC freq'], errors='coerce')

# Drop object columns for imputation
object_columns_glm = data_extracted_features_impute_glm.select_dtypes(include=['object']).columns
data_extracted_features_impute_glm_copy = data_extracted_features_impute_glm.drop(columns=object_columns_glm)

# Set up the imputer with LinearRegression estimator
imputer_glm = IterativeImputer(estimator=LinearRegression(), min_value=0,  random_state=0, max_iter=100)

# Fit the imputer to your data and transform it
data_extracted_features_imputed_glm = imputer_glm.fit_transform(data_extracted_features_impute_glm_copy)
data_extracted_features_imputed_glm = pd.DataFrame(data_extracted_features_imputed_glm, columns=data_extracted_features_impute_glm_copy.columns)

# Add back the object columns
for column in object_columns_glm:
    data_extracted_features_imputed_glm[column] = data_extracted_features_impute_glm[column]

# Create dictioniary to store imputed values to be exported to Excel
dict_imputed_glm_values = {
    'Feature': [],
    'Index': [],
    'Original Value': [],
    'Imputed Value': []
}

# Print imputed values for each column
for column_glm in data_extracted_features_impute_glm_copy.columns:
    missing_mask_column_glm = data_extracted_features_impute_glm_copy[column_glm].isnull()
    imputed_missing_mask_column_glm = data_extracted_features_imputed_glm[column_glm].isnull()
    imputed_values_mask_column_glm = missing_mask_column_glm & ~imputed_missing_mask_column_glm
    imputed_values_column_glm = data_extracted_features_impute_glm_copy.loc[imputed_values_mask_column_glm, column_glm]
    imputed_value_glm = data_extracted_features_imputed_glm.loc[imputed_values_mask_column_glm, column_glm]
    if not imputed_values_column_glm.empty:
        if not os.path.exists('run_1/qc/tables/GLM_imputation'):
            os.makedirs('run_1/qc/tables/GLM_imputation')
        imputed_value_pairs_glm = [(original_glm, imputed_glm) for original_glm, imputed_glm in zip(imputed_values_column_glm, imputed_value_glm)]
        print(f"Imputed values for {column_glm}:")
        formatted_output = "\n".join([f"Index: {index_glm + 2}\nOriginal Value: {original_glm} --> Imputed value: {imputed_glm}" for index_glm, (original_glm, imputed_glm) in zip(imputed_values_column_glm.index, imputed_value_pairs_glm)])
        print(formatted_output + "\n")
        # Store imputed values in dictionary
        indices_glm = imputed_values_mask_column_glm[imputed_values_mask_column_glm].index
        dict_imputed_glm_values['Feature'].extend([column_glm] * len(indices_glm))
        dict_imputed_glm_values['Index'].extend(indices_glm + 2)  # Adding 2 to match your index adjustment
        dict_imputed_glm_values['Original Value'].extend(imputed_values_column_glm.fillna('nan').tolist())
        dict_imputed_glm_values['Imputed Value'].extend(imputed_value_glm.tolist())
        #print(dict_imputed_values)

# Export imputed values to Excel
imputed_df_glm = pd.DataFrame(dict_imputed_glm_values)  
imputed_df_glm.to_csv(f"run_1/qc/tables/GLM_imputation/GLM_imputed_values_0_FALSE_100.csv", index=False)


In [ ]:
#This uses the MICE imputed dataset

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from pycombat import pycombat
from sklearn.model_selection import StratifiedShuffleSplit
from matplotlib import pyplot as plt

# Make copy of data to not alter real data
data_extracted_features_impute = data_extracted_features.copy()

# Replace '>' values with NaN in 'EPSC freq' column
mask = data_extracted_features_impute['EPSC freq'].notna() & data_extracted_features_impute['EPSC freq'].str.contains(">")
data_extracted_features_impute.loc[mask, 'EPSC freq'] = np.nan

# Convert 'EPSC freq' column to numeric
data_extracted_features_impute['EPSC freq'] = pd.to_numeric(data_extracted_features_impute['EPSC freq'])

#Define imputer
imputer = IterativeImputer(random_state=0, min_value=0, sample_posterior=False, max_iter=100, tol=0.001)

# Drop object columns for imputation. The columns which contain categorical values will be dropped for the MICE imputation to work.
# These columns will be added back after imputation.
object_columns = data_extracted_features_impute.select_dtypes(include=['object']).columns
data_extracted_features_impute_copy = data_extracted_features_impute.drop(columns=object_columns)

#Impute missing values
data_extracted_features_imputed = imputer.fit_transform(data_extracted_features_impute_copy)
data_extracted_features_imputed = pd.DataFrame(data_extracted_features_imputed, columns=data_extracted_features_impute_copy.columns)

#Make another copy to not alter real data
data_extracted_features_imputed_combat = data_extracted_features_imputed.copy()

data_extracted_features_imputed_combat['Batch number'] = data_extracted_features_imputed_combat['Batch number'].astype(str)
data_extracted_features_imputed_combat_categorical = data_extracted_features_imputed_combat.select_dtypes(include=['object'])
data_extracted_features_imputed_combat_numeric = data_extracted_features_imputed_combat.select_dtypes(include=['float64', 'int64'])   

#PyCombat doesn't use a pandas dataframe, but a numpy array.
combat = pycombat.Combat()

#These variables are used in SVM. The lines above make sure that the correct data is used
data_extracted_features_imputed_combat_categorical = data_extracted_features_imputed_combat.select_dtypes(include=['object'])
data_extracted_features_imputed_combat_numeric_transformed = combat.fit_transform(data_extracted_features_imputed_combat_numeric.to_numpy(), b=data_extracted_features_imputed_combat['Batch number']) 
data_extracted_features_imputed_combat_numeric_transformed = pd.DataFrame(data_extracted_features_imputed_combat_numeric_transformed, columns=data_extracted_features_imputed_combat_numeric.columns)

data = data_extracted_features_imputed_combat_numeric_transformed.copy()

# Separate features and target variable
X = data.drop('Div calculated', axis=1)
y = data['Div calculated']

# Split the data into training and testing sets
# Random_state ensures that data is divided the same every time

# Initialize StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.1, random_state=42)

svm = SVC(kernel='linear', C=1.0, random_state=42)

# Split the data
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    y_train = y_train.astype(int)
    y_test = y_test.astype(int)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy:.2f}')
    print('Classification Report:')
    print(classification_report(y_test, y_pred))
    confusion_mat = confusion_matrix(y_test, y_pred)
    print('Confusion Matrix:')
    print(confusion_matrix(y_test, y_pred))

    sns.heatmap(confusion_mat)
    plt.show()

    
# Check if the split is done correctly
# print("Training Features Shape:", X_train.shape)
# print("Testing Features Shape:", X_test.shape)
# print("Training Target Shape:", y_train.shape)
# print("Testing Target Shape:", y_test.shape)

